<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/Reddit_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

In [5]:
!pip install wrds datetime psaw
import pandas as pd, numpy as np, wrds, datetime, sqlite3, requests, os, re
import datetime as dt
from psaw import PushshiftAPI

# Read datasets

In [6]:
option_list = pd.read_sas("/content/drive/MyDrive/option_ticker.sas7bdat",encoding="utf-8")

In [7]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [8]:
om_name = (db.get_table('optionm','secnmd')
                .groupby(['secid','ticker','issuer'])['effect_date']
                .agg(['min','max'])
                .reset_index().rename({'min':'start','max':'end'},axis=1))

In [9]:
om_name['ticker'] = np.where(om_name['ticker']=='ZZZZ',np.NaN,om_name['ticker'])

Unique ticker list from OptionMetrics

In [10]:
ticker = list(om_name.ticker.dropna().unique())

---
# *Don't run here if file already saved.*
---

## Grab Reddit submissions using Pushshift API



In [11]:
api = PushshiftAPI()

We only examine posts up to 2020.

In [12]:
if not os.path.exists('/content/drive/MyDrive/submission.parquet.gzip'):
    field = ['id','created','created_utc','num_comments','score',
            'upvote_ratio','full_link','title','selftext','link_flair_text']
    end_epoch = int(dt.datetime(2021,1,1).timestamp())
    gen = api.search_submissions(subreddit='wallstreetbets',
                                before=end_epoch)
    submission = pd.DataFrame([post.d_ for post in gen])[field]
    submission['date'] = pd.to_datetime(submission['created'], unit='s', utc=True)
    submission['date_utc'] = pd.to_datetime(submission['created_utc'], unit='s', utc=True)
    col = submission.columns.to_list()
    col = col[0:1] + col[-2:] + col[3:-2]
    submission = submission[col]
    #submission = pd.DataFrame([post.d_ for post in gen])
    submission.to_parquet('/content/drive/MyDrive/submission.parquet.gzip', compression='gzip')

---

In [13]:
submission = pd.read_parquet('/content/drive/MyDrive/submission.parquet.gzip')

In [14]:
submission.set_index(['id'],inplace=True)

# Link ticker to WSB posts

In [52]:
def find_ticker(text):
    global ticker
    a_set = set(text.split())
    b_set = set(ticker).union(set('$' + tic for tic in ticker))
    hit = a_set.intersection(b_set)
    if len(a_set.intersection(b_set)) > 0:
        return [s[1:] if s.startswith('$') else s  for s in hit]
    else:
        return []            

In [ ]:
submission['title_ticker'] = submission['title'].apply(find_ticker)
submission['selftext_ticker'] = submission['selftext'].apply(find_ticker)

In [ ]:
submission.to_parquet('/content/drive/MyDrive/submission_ticker.parquet.gzip', compression='gzip')

Variables to capture from posts

* Number of posts
* Combined rewards
* Comments
* Score